## ML HW2 手把手教學 


In [ ]:
import numpy as np
import pandas as pd

### Probabilistic generative model


In [ ]:
import numpy as np
import math
import pandas as pd

dim = 106

Same as logictic regression

In [ ]:
def load_data():
    x_train = pd.read_csv('X_train')
    x_test = pd.read_csv('X_test')

    x_train = x_train.values
    x_test = x_test.values

    y_train = pd.read_csv('Y_train', header = None)
    y_train = y_train.values
    y_train = y_train.reshape(-1)


    return x_train, y_train, x_test

def sigmoid(z):
    res = 1 / (1.0 + np.exp(-z))
    return np.clip(res, 1e-6, 1-1e-6)

參考 [上課投影片](https://drive.google.com/file/d/1WKjqkJVPIxYh1REbzy6HeoGfZj-mj6NJ/view) P18 and P23



In [ ]:
def train(x_train, y_train):
   
    mu1 = np.zeros((dim,))
    mu2 = np.zeros((dim,))
    # implement probalistic generative model

    idx_1 = np.argwhere(y_train == 1).flatten()
    idx_2 = np.argwhere(y_train == 0).flatten()
    class1 = x_train[idx_1]
    class2 = x_train[idx_2]

    cnt1 = idx_1.shape[0]
    cnt2 = idx_2.shape[0]

    mu1 = np.mean(class1, axis=0)
    mu2 = np.mean(class2, axis=0)

    sigma1 = np.zeros((dim, dim))
    sigma2 = np.zeros((dim, dim))
     
    for i in range(cnt1):
      m = (class1[i] - mu1).reshape(dim,1)
      sigma1 += np.matmul(m, np.transpose(m))
    for i in range(cnt2):
      m = (class2[i] - mu2).reshape(dim,1)
      sigma2 += np.matmul(m, np.transpose(m))
    
    sigma1 = sigma1 / cnt1
    sigma2 = sigma2 / cnt2
    share_sigma = (float(cnt1)/x_train.shape[0]) * sigma1 + (float(cnt2)/x_train.shape[0]) * sigma2

    return mu1, mu2, share_sigma, cnt1, cnt2

參考 [上課投影片](https://drive.google.com/file/d/1WKjqkJVPIxYh1REbzy6HeoGfZj-mj6NJ/view) P33



In [ ]:
def predict(x_test, mu1, mu2, share_sigma, N1, N2):
    sigma_inverse = np.linalg.inv(share_sigma)

    w = np.dot( (mu1-mu2), sigma_inverse)
    b = (-0.5) * np.dot(np.dot(mu1.T, sigma_inverse), mu1) + (0.5) * np.dot(np.dot(mu2.T, sigma_inverse), mu2) + np.log(float(N1)/N2)

    z = np.dot(w, x_test.T) + b
    pred = sigmoid(z)
    return pred

Predict your results.



In [ ]:

x_train,y_train,x_test = load_data()

mu1, mu2, shared_sigma, N1, N2 = train(x_train, y_train)


y = predict(x_train, mu1, mu2, shared_sigma, N1, N2)
y = np.around(y)

result = (y_train == y)

print('Train acc = %f' % (float(result.sum()) / result.shape[0]))

y_test = predict(x_test, mu1, mu2, shared_sigma, N1, N2)
y_test = np.around(y_test)


In [ ]:
import csv
with open('predict.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['id','label'])
    for i in range(int(y_test.shape[0])):
      writer.writerow( [i + 1, int(y_test[i])] )

### Tip for math problem
[p1](https://people.eecs.berkeley.edu/~jrs/189/exam/mids14.pdf)  
[p2&3](https://people.eecs.berkeley.edu/~jordan/courses/260-spring10/other-readings/chapter13.pdf)  
[p3](https://stats.stackexchange.com/questions/351549/maximum-likelihood-estimators-multivariate-gaussian)